##### Import relevant packages

In [86]:
import pandas as pd
import matplotlib

##### Reference similarweb .csv files and read to DataFrames (need to be more programmatic)

In [87]:
feb_data_file = 'similarweb_feb.csv'
mar_data_file = 'similarweb_march.csv'
apr_data_file = 'similarweb_april.csv'

In [88]:
df_feb = pd.read_csv(feb_data_file)
df_mar = pd.read_csv(mar_data_file)
df_apr = pd.read_csv(apr_data_file, encoding='iso-8859-1')

##### Preview the top 5 rows of the February DataFrame

In [89]:
df_apr.head()

,Account Name,Name,Average Visit Duration,Website Category,2nd Traffic Country,Bounce Rate,Category Rank,Desktop Visits Share,Direct Visits Share,Display Ads Visits Share,...,Mobile Web Visits Share,Monthly Unique Visitors,Pages Per Visit,Total Monthly Visits,Total Visits MoM Growth,Top Traffic Country,Social Visits Share,Paid Search Visits Share,Account ID,Snapshot Date
0,Meez Culinary Solutions LLC,getmeez.com,NaN,NaN,NaN,NaN,NaN,0.00,NaN,NaN,...,0.00,0,NaN,NaN,-100.00,NaN,NaN,NaN,0012K00001fHHQN,4/1/2020
1,Cheetah,gocheetah.com,00:01:23,NaN,Israel,48.51,NaN,75.06,49.56,NaN,...,24.94,19672,2.17,28943.0,57.65,United States,9.13,0.96,0012K00001hjtaz,4/1/2020
2,Wunderlich Securities Inc,wunderlichsecurities.com,NaN,NaN,NaN,NaN,NaN,0.00,NaN,NaN,...,0.00,0,NaN,NaN,-100.00,NaN,NaN,NaN,0016A00000n9zBj,4/1/2020
3,The Grand Island Independent,theindependent.com,00:01:56,News and Media,Canada,61.63,9646.0,33.58,30.49,0.2,...,66.42,198054,2.80,500284.0,40.42,United States,9.94,NaN,0016A00000n9yFu,4/1/2020
4,Memphis Communications Corp,memphiscommunications.net,00:04:13,NaN,Slovakia,61.62,NaN,100.00,32.79,NaN,...,0.00,870,2.69,1686.0,63.40,United States,0.27,NaN,0016A00000n9vjn,4/1/2020


##### Reorder columns to most logical sequencing (not required)

In [90]:
columns_ordered = ['Account ID','Account Name', 'Name', 'Domain', 'Website Category','Category Rank',
                   'Global Rank', 'Top Traffic Country','2nd Traffic Country', 'Has Data','Total Monthly Visits',
                   'Monthly Unique Visitors','Bounce Rate', 'Pages Per Visit', 'Average Visit Duration',
                   'Total Visits MoM Growth','Desktop Visits Share', 'Mobile Web  Visits Share',
                   'Direct Visits Share','Display Ads Visits Share','Mail Visits Share',
                   'Paid Search Visits Share', 'Social Visits Share','Snapshot Date']
df_feb = df_feb[columns_ordered]
df_mar = df_mar[columns_ordered]
df_apr = df_apr[columns_ordered]

##### Fill all #NAs with 0s for Monthly Visits and Paid Search/Display Ads columns

In [91]:
df_feb['Total Monthly Visits'].fillna(0, inplace=True)
df_feb['Paid Search Visits Share'].fillna(0, inplace=True)
df_feb['Display Ads Visits Share'].fillna(0, inplace=True)

df_mar['Total Monthly Visits'].fillna(0, inplace=True)
df_mar['Paid Search Visits Share'].fillna(0, inplace=True)
df_mar['Display Ads Visits Share'].fillna(0, inplace=True)

df_apr['Total Monthly Visits'].fillna(0, inplace=True)
df_apr['Paid Search Visits Share'].fillna(0, inplace=True)
df_apr['Display Ads Visits Share'].fillna(0, inplace=True)

##### Compute Total Ad Spend Visits Share (Paid Search + Display Ads) then calculate # of visits from Total Monthly Visits

In [92]:
df_feb['Total Ad Spend Visits Share'] = df_feb['Paid Search Visits Share'] + df_feb['Display Ads Visits Share']
df_feb['Total Ad Spend Visits'] = df_feb['Total Monthly Visits']*df_feb['Total Ad Spend Visits Share']/100

df_mar['Total Ad Spend Visits Share'] = df_mar['Paid Search Visits Share'] + df_mar['Display Ads Visits Share']
df_mar['Total Ad Spend Visits'] = df_mar['Total Monthly Visits']*df_mar['Total Ad Spend Visits Share']/100

df_apr['Total Ad Spend Visits Share'] = df_apr['Paid Search Visits Share'] + df_apr['Display Ads Visits Share']
df_apr['Total Ad Spend Visits'] = df_apr['Total Monthly Visits']*df_apr['Total Ad Spend Visits Share']/100

###### Slim down DataFrame for only relevant columns to be output to .csv

In [93]:
df_feb_slim = df_feb[['Account ID', 'Account Name', 'Domain', 'Website Category', 'Total Monthly Visits',
                     'Total Visits MoM Growth','Total Ad Spend Visits Share','Total Ad Spend Visits']]

df_mar_slim = df_mar[['Account ID', 'Account Name', 'Domain', 'Website Category', 'Total Monthly Visits',
                     'Total Visits MoM Growth','Total Ad Spend Visits Share','Total Ad Spend Visits']]

df_apr_slim = df_apr[['Account ID', 'Account Name', 'Domain', 'Website Category', 'Total Monthly Visits',
                     'Total Visits MoM Growth','Total Ad Spend Visits Share','Total Ad Spend Visits']]

###### Left join March and February, add suffixes to column names and drop redundant columns

In [94]:
merged_apr_mar = df_apr_slim.merge(df_mar_slim,on=['Account ID'],how='left',suffixes=('_apr', '_mar'))

df_feb_slim.columns = df_feb_slim.columns.map(lambda x: str(x) + '_feb' if x != 'Account ID' else x)

merged_apr_mar_feb = merged_apr_mar.merge(df_feb_slim,on=['Account ID'],how='left')

merged_apr_mar_feb.drop(columns=['Account Name_feb', 'Domain_feb', 'Website Category_feb','Account Name_mar', 'Domain_mar', 'Website Category_mar'], axis=1, inplace=True)

##### Convert Account ID (15-char) to 18-char ID

In [95]:
account_map_file = 'account_id_map.csv'
df_map = pd.read_csv(account_map_file, encoding='iso-8859-1')


df_map.drop(columns=['Parent Account ID'], axis=1, inplace=True)

merged_apr_mar_feb = merged_apr_mar_feb.merge(df_map, on=['Account ID'], how='left')

cols = list(merged_apr_mar_feb.columns)
cols.insert(0,cols.pop(cols.index('Account ID (18)')))
merged_apr_mar_feb = merged_apr_mar_feb.loc[:,cols]


##### Export merged DataFrame to .csv file on local repository

In [96]:
merged_apr_mar_feb.to_csv('feb_march_apr_similarweb.csv')